Write a Python function called rentals_month that retrieves rental data for a given month and year (passed as parameters) from the Sakila database as a Pandas DataFrame. The function should take in three parameters:

* engine: an object representing the database connection engine to be used to establish a connection to the Sakila database.
* month: an integer representing the month for which rental data is to be retrieved.
* year: an integer representing the year for which rental data is to be retrieved.

The function should execute a SQL query to retrieve the rental data for the specified month and year from the rental table in the Sakila database, and return it as a pandas DataFrame.



In [9]:
import mysql.connector
import getpass
password = getpass.getpass()
import pandas as pd


engine = mysql.connector.connect(
    host='localhost',
    user='root',
    password=password,
    database='sakila'
)

In [15]:
def rental_months(engine, month, year):
    cursor = engine.cursor()

    query = """
    SELECT *,
           DATE_FORMAT(rental_date, '%m') as rental_month,
           DATE_FORMAT(rental_date, '%Y') as rental_year
    FROM rental
    WHERE DATE_FORMAT(rental_date, '%m') = %s AND DATE_FORMAT(rental_date, '%Y') = %s
    """
    
    cursor.execute(query, (month, year))
    query_1 = cursor.fetchall()

    data = []

    for row in query_1:
        data.append(row)

    cursor.close()

    df = pd.DataFrame(data, columns=[
        "rental_id", "rental_date", "inventory_id", "customer_id", 
        "return_date", "staff_id", "last_update", "rental_month", "rental_year"
    ])
    return df

In [87]:
password = getpass.getpass()
engine = mysql.connector.connect(
    host='localhost',
    user='root',
    password=password,
    database='sakila'
)

df = rental_months(engine, '06', '2005')
engine.close()

df.head(10)

df.groupby("customer_id").agg({"rental_id": "count"})

,rental_id
customer_id,
1,7
2,1
3,4
4,6
5,5
...,...
595,2
596,2
597,3


Develop a Python function called *rental_count_month* that takes the DataFrame provided by rentals_month as input along with the month and year and returns a new DataFrame containing the number of rentals made by each customer_id during the selected month and year.

The function should also include the month and year as parameters and use them to name the new column according to the month and year, for example, if the input month is 05 and the year is 2005, the column name should be "rentals_05_2005".

*Hint: Consider making use of pandas groupby()*

In [92]:
def rental_count_month(engine, month, year):
    df = rental_months(engine, month, year)
    df_filtered = df[(df["rental_month"] == month) & (df["rental_year"] == year)]
    df_grouped = df_filtered.groupby("customer_id").agg({"rental_id": "count"})

    new_column_names = {col: f"rentals_{month}_{year}" for col in df_grouped.columns}

    df_2 = df_grouped.rename(columns=new_column_names)
    return df_2

In [107]:
engine = mysql.connector.connect(
    host='localhost',
    user='root',
    password=password,
    database='sakila'
)

rental_count_month(engine, '06', '2005')

,rentals_06_2005
customer_id,
1,7
2,1
3,4
4,6
5,5
...,...
595,2
596,2
597,3


Create a Python function called compare_rentals that takes two DataFrames as input containing the number of rentals made by each customer in different months and years. The function should return a combined DataFrame with a new 'difference' column, which is the difference between the number of rentals in the two months.

In [113]:
df_1 = rental_count_month(engine, '06', '2005')
df_2 = rental_count_month(engine, '05', '2005')

def compare_rentals(df_1, df_2):
    df_3 = pd.concat([df_1, df_2], axis = 1)
    df_3['difference'] = df_3.apply(lambda row: row.rentals_06_2005 - row.rentals_05_2005, axis = 1)
    df_final = round(df_3, 0)
    return df_final

In [ ]:
df_1 = rental_count_month(engine, '06', '2005')
df_2 = rental_count_month(engine, '05', '2005')

def compare_rentals(df_1, df_2):
    df_3 = pd.concat([df_1, df_2], axis = 1)
    df_3['difference'] = df_3.apply(lambda row: row.rentals_06_2005 - row.rentals_05_2005, axis = 1)
    df_final = round(df_3, 0)
    return df_final

df_result = compare_rentals(df_1, df_2)
df_result.head()

,rentals_06_2005,rentals_05_2005,difference
customer_id,,,
1,7.0,2.0,5.0
2,1.0,1.0,0.0
3,4.0,2.0,2.0
4,6.0,NaN,NaN
5,5.0,3.0,2.0
